In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import env
import acquire as a
from os.path import isfile
from importlib import reload

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

In [3]:
# def acquire_cohorts()->pd.DataFrame:
#     if isfile('cohort_logs.csv'):
#         return pd.read_csv('cohort_logs.csv',index_col=[0])
#     query = '''SELECT id AS cohort, name AS cohort_name , start_date, end_date, program_id FROM cohorts'''
#     url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
#     df = pd.read_sql(query,url)
#     df.index = df.cohort
#     return cohorts_df

In [6]:
# def acquire_curriculum_data():
#     '''This function gets curriculum data from the local repo, which is stored
#     as a .txt file'''
#     #Get the data from text file on local repo
#     df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])
#     df.date = pd.to_datetime(df.date + ' ' + df.time)
#     df = df.drop(columns=['time'])
#     df = df.set_index(df['date'])
#     df = df.drop(columns=['date'])               
#     return df

In [10]:
# def clean_cohort_logs(df):
#     #Get csv from file
#     df = pd.read_csv('cohort_logs.csv', index_col=[0])
#     #Changing date, start and end dates to datetime fields
#     df["date"]= pd.to_datetime(df["date"])
#     df["start_date"]= pd.to_datetime(df["start_date"])
#     df["end_date"]= pd.to_datetime(df["end_date"])
#     #Setting index as date time
#     #Set datetime index
#     df = df.set_index(df.date)
#     #Creating new column for program length
#     df['program_length'] = df.end_date - df.start_date
#     #Create column with program name
#     #Change data types as needed
#     df["program_id"]= df["program_id"].astype(str)
#     #Copying these values to a new column program name
#     df['program_name'] = df['program_id']
#     #Mapping the real names of the program
#     df["program_name"] = df["program_name"].map({'1.0':'full stack PHP','2.0':'full stack Java','3.0':'data science','4.0':'front end'})
    
#     return df

In [11]:
# def wrangle_curriculum_access()->pd.DataFrame:
#     curriculum_df = acquire_curriculum_data()
#     cohort_df = acquire_cohorts()
#     cohort_df = clean_cohort_logs(cohort_df)
#     curriculum_df = curriculum_df.join(cohort_df,on='cohort',how='left',lsuffix='_id')
#     curriculum_df = curriculum_df.drop(columns=['cohort_id','cohort','program_id'])
#     curriculum_df.program_name = curriculum_df.program_name.astype('category')
#     return curriculum_df

In [2]:
reload(a)
curriculum_df = a.wrangle_curriculum_access()
curriculum_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java


In [3]:
curriculum_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   page            900222 non-null  object         
 1   id              900223 non-null  int64          
 2   ip              900223 non-null  object         
 3   cohort_name     847330 non-null  object         
 4   start_date      847330 non-null  datetime64[ns] 
 5   end_date        847330 non-null  datetime64[ns] 
 6   program_length  847330 non-null  timedelta64[ns]
 7   program_name    847330 non-null  category       
dtypes: category(1), datetime64[ns](2), int64(1), object(3), timedelta64[ns](1)
memory usage: 55.8+ MB


In [2]:
# def acquire_cohorts():
#     if isfile('cohorts.csv'):
#         return pd.read_csv('cohorts.csv',index_col=[0])
#     query = '''SELECT id AS cohort, name AS cohort_name , start_date, end_date, program_id FROM cohorts'''
#     url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
#     df = pd.read_sql(query,url)
#     df.index = df.cohort
#     return df

In [3]:
# df = acquire_cohorts()

NameError: name 'isfile' is not defined

In [2]:
# # Import .txt file and convert it to a DataFrame object
# df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [3]:
# def acquire_curriculum_data():
#     #Get the data from text file on local repo
#     df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])
#     return df

In [4]:
# df = acquire_curriculum_data()

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [5]:
#Total Users
curriculum_df.id.nunique()

981

In [6]:
curriculum_df.id.value_counts().tail(20)

348    3
176    3
177    3
852    2
97     2
573    2
574    2
649    1
940    1
165    1
879    1
652    1
592    1
619    1
593    1
66     1
163    1
918    1
212    1
952    1
Name: id, dtype: int64

In [20]:
curriculum_df.ip.nunique()

5531

In [25]:
curriculum_df.ip.value_counts()

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64

In [29]:
curriculum_df.cohort.value_counts()

28.0     84031
33.0     40730
29.0     38096
62.0     37109
53.0     36902
24.0     35636
57.0     33844
56.0     33568
51.0     32888
59.0     32015
22.0     30926
58.0     29855
32.0     29356
23.0     28534
52.0     28033
26.0     27749
34.0     26538
25.0     25586
31.0     25359
132.0    23691
55.0     21582
27.0     20743
61.0     17713
134.0    16623
135.0    16397
133.0    14715
14.0      9587
1.0       8890
137.0     8562
21.0      7444
138.0     7276
17.0      4954
13.0      2845
18.0      2158
8.0       1712
139.0     1672
19.0      1237
16.0       755
15.0       691
7.0        598
12.0       302
11.0       253
2.0         93
6.0         72
9.0          5
4.0          4
5.0          1
Name: cohort, dtype: int64

In [32]:
curriculum_df.isnull().sum()

date          0
time          0
page          1
id            0
cohort    52893
ip            0
dtype: int64

In [8]:
curriculum_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   page            900222 non-null  object         
 1   id              900223 non-null  int64          
 2   ip              900223 non-null  object         
 3   cohort_name     847330 non-null  object         
 4   start_date      847330 non-null  datetime64[ns] 
 5   end_date        847330 non-null  datetime64[ns] 
 6   program_length  847330 non-null  timedelta64[ns]
 7   program_name    847330 non-null  category       
dtypes: category(1), datetime64[ns](2), int64(1), object(3), timedelta64[ns](1)
memory usage: 55.8+ MB


In [6]:
# def acquire_cohort_logs(user=env.user, password=env.password, host=env.host):
#     '''
#     This function queries the Codeup MySQL curriculum_logs database and returns a dataframe
#     '''
#     url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
#     query = '''
#     SELECT date, path as endpoint, user_id, cohort_id, name, start_date, end_date, ip as source_ip, program_id
#         FROM logs l
#         LEFT JOIN cohorts c ON l.cohort_id = c.id;
#     '''
#     df = pd.read_sql(query, url)
#     #Save file to csv
#     df.to_csv('cohort_logs.csv')
#     return df

In [4]:
#Taking all activity for students, not staff
student_df = curriculum_df[curriculum_df.cohort_name != "Staff"]
student_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java


In [5]:
#Taking all activity from students active in course(from start date to end date)
active_df = student_df[(student_df.end_date >= student_df.index) & (student_df.start_date <= student_df.index)]
active_df = active_df[active_df['page'] != '/']
active_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:41,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:46,javascript-i/conditionals,3,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:48,javascript-i/functions,3,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:59,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java


In [6]:
#617k active student log entries
active_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 616912 entries, 2018-01-26 09:56:24 to 2021-04-21 16:41:51
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   page            616911 non-null  object         
 1   id              616912 non-null  int64          
 2   ip              616912 non-null  object         
 3   cohort_name     616912 non-null  object         
 4   start_date      616912 non-null  datetime64[ns] 
 5   end_date        616912 non-null  datetime64[ns] 
 6   program_length  616912 non-null  timedelta64[ns]
 7   program_name    616912 non-null  category       
dtypes: category(1), datetime64[ns](2), int64(1), object(3), timedelta64[ns](1)
memory usage: 38.2+ MB


In [7]:
#Aggregating pages by student
pages_by_student = active_df.groupby(['id'])['page'].value_counts()
pages_by_student

id   page                                            
2    java-iii                                            50
     mysql                                               32
     java-ii                                             30
     spring                                              29
     jquery                                              28
                                                         ..
981  examples/css/bootstrap-grids.html                    3
     appendix/professional-development/t-block-resume     1
     examples/bootstrap-grid/assets.zip                   1
     html-css/css-ii                                      1
     javascript-ii                                        1
Name: page, Length: 88169, dtype: int64

In [8]:
#make a df
pages_by_student =pd.DataFrame(pages_by_student)
pages_by_student.head()

page
id page          
2  java-iii    50
   mysql       32
   java-ii     30
   spring      29
   jquery      28

In [9]:
pages_by_student.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 88169 entries, (2, 'java-iii') to (981, 'javascript-ii')
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   page    88169 non-null  int64
dtypes: int64(1)
memory usage: 1.0+ MB


In [10]:
pages_by_student.columns = ['page_count']
pages_by_student = pages_by_student.reset_index()
pages_by_student = pages_by_student.groupby('id').sum('page_count')
pages_by_student.nsmallest(n=50, columns='page_count')

,page_count
id,
832,2
278,3
539,4
956,4
388,7
812,7
679,10
941,10
572,11


In [22]:
#Looking at 832, Very little activity, in the middle of the course too, full stack Java Jupiter
user_832 = active_df[active_df.id == 832]
user_832.page.value_counts()

javascript-i    1
html-css        1
Name: page, dtype: int64

In [32]:
user_832

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2020-12-07 14:22:38,javascript-i,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,full stack Java
2020-12-07 14:22:39,html-css,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,full stack Java


In [34]:
curriculum_df[curriculum_df.id == 832]

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2020-12-07 14:22:00,/,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,full stack Java
2020-12-07 14:22:38,javascript-i,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,full stack Java
2020-12-07 14:22:39,html-css,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,full stack Java


In [23]:
#Looking at 278
user_278 = active_df[active_df.id == 278]
user_278.page.value_counts()

java-ii/arrays         2
java-ii/collections    1
Name: page, dtype: int64

In [24]:
user_278

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-09-27 14:47:37,java-ii/arrays,278,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,135 days,full stack Java
2018-09-27 14:58:48,java-ii/arrays,278,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,135 days,full stack Java
2018-09-27 14:59:07,java-ii/collections,278,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,135 days,full stack Java


In [25]:
#Looking at 539
user_539 = active_df[active_df.id == 539]
user_539.page.value_counts()

toc                      1
html-css                 1
html-css/introduction    1
html-css/elements        1
Name: page, dtype: int64

In [35]:
#User 539 may not have stayed in program
user_539

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2019-11-04 16:19:34,toc,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,full stack Java
2019-11-04 16:54:02,html-css,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,full stack Java
2019-11-04 16:54:38,html-css/introduction,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,full stack Java
2019-11-04 16:54:45,html-css/elements,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,full stack Java


In [27]:
#Looking at 956
user_956 = active_df[active_df.id == 956]
user_956.page.value_counts()

javascript-i/introduction/operators          2
toc                                          1
javascript-i/introduction/primitive-types    1
Name: page, dtype: int64

In [29]:
#Oberon Full Stack Java, may not have stayed in program
user_956

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2021-04-15 14:54:10,toc,956,162.200.114.251,Oberon,2021-04-12,2021-10-01,172 days,full stack Java
2021-04-15 14:55:05,javascript-i/introduction/operators,956,162.200.114.251,Oberon,2021-04-12,2021-10-01,172 days,full stack Java
2021-04-15 14:56:04,javascript-i/introduction/primitive-types,956,162.200.114.251,Oberon,2021-04-12,2021-10-01,172 days,full stack Java
2021-04-15 15:32:22,javascript-i/introduction/operators,956,162.200.114.251,Oberon,2021-04-12,2021-10-01,172 days,full stack Java


In [30]:
#Looking at 388
user_388 = active_df[active_df.id == 388]
user_388.page.value_counts()

html-css/elements    5
html-css             1
html-css/forms       1
Name: page, dtype: int64

In [31]:
#User 388 may have left the program after the first day
user_388

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2019-03-19 09:50:23,html-css,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java
2019-03-19 09:50:28,html-css/elements,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java
2019-03-19 10:04:11,html-css/elements,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java
2019-03-19 10:19:32,html-css/elements,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java
2019-03-19 11:11:51,html-css/forms,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java
2019-03-19 11:12:02,html-css/elements,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java
2019-03-19 12:19:23,html-css/elements,388,97.105.19.58,Andromeda,2019-03-18,2019-07-30,134 days,full stack Java


In [38]:
#Looking at 812
user_812 = active_df[active_df.id == 812]
user_812.page.value_counts()

html-css/elements                          2
html-css/css-i/selectors-and-properties    1
html-css/css-i/introduction                1
html-css/css-i                             1
html-css/forms                             1
html-css                                   1
Name: page, dtype: int64

In [39]:
user_812

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2020-11-08 01:45:34,html-css/css-i/selectors-and-properties,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java
2020-11-08 01:45:41,html-css/elements,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java
2020-11-08 01:45:56,html-css/css-i/introduction,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java
2020-11-08 01:46:01,html-css/css-i,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java
2020-11-08 01:46:04,html-css/forms,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java
2020-11-08 01:46:06,html-css/elements,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java
2020-11-08 01:46:07,html-css,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,full stack Java


In [40]:
#Looking at 679
user_679 = active_df[active_df.id == 679]
user_679.page.value_counts()

1-fundamentals/1.1-intro-to-data-science    3
1-fundamentals/AI-ML-DL-timeline.jpg        3
1-fundamentals/modern-data-scientist.jpg    3
13-advanced-topics/1-tidy-data              1
Name: page, dtype: int64

In [41]:
user_679

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2020-07-13 14:39:21,13-advanced-topics/1-tidy-data,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-13 14:39:36,1-fundamentals/1.1-intro-to-data-science,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-13 14:39:37,1-fundamentals/AI-ML-DL-timeline.jpg,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-13 14:39:37,1-fundamentals/modern-data-scientist.jpg,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-13 15:49:31,1-fundamentals/1.1-intro-to-data-science,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-13 15:49:32,1-fundamentals/modern-data-scientist.jpg,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-13 15:49:32,1-fundamentals/AI-ML-DL-timeline.jpg,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-14 08:05:15,1-fundamentals/1.1-intro-to-data-science,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science
2020-07-14 08:05:15,1-fundamentals/AI-ML-DL-timeline.jpg,679,24.28.146.155,Darden,2020-07-13,2021-01-12,183 days,data science


### Many of the users with small page views may have left the program (dates of page views correspond to the begnning of the course. I will reshift those to students who had at least 50 page views

In [50]:
#Students with at least 50 page views, which are 
pages_by_student[pages_by_student >= 50].nsmallest(n=10, columns='page_count')

,page_count
id,
56,50.0
46,51.0
543,51.0
962,55.0
36,56.0
858,59.0
57,61.0
45,62.0
975,64.0


In [48]:
#Looking at 56
user_56 = active_df[active_df.id == 56]
user_56.page.value_counts()

spring                                         7
mysql                                          5
mysql/databases                                5
appendix                                       4
spring/fundamentals/relationships              3
spring/fundamentals/form-model-binding         3
spring/fundamentals/services                   3
spring/fundamentals/security/authentication    2
html-css                                       2
spring/fundamentals/repositories               2
mysql/users                                    2
java-iii                                       2
spring/fundamentals/controllers                2
appendix/further-reading/spring/seeder         1
mysql/basic-statements                         1
spring/fundamentals/views                      1
html-css/elements                              1
mysql/introduction                             1
spring/extra-features/file-upload              1
appendix/capstone-workbook                     1
spring/setup        

In [49]:
user_56

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-29 09:40:38,spring/fundamentals/relationships,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 09:40:41,spring/fundamentals/form-model-binding,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 09:40:44,spring/fundamentals/controllers,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 09:40:48,spring,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 12:31:31,java-iii,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 12:31:35,mysql,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 12:31:40,mysql/databases,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 12:32:25,mysql/users,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java
2018-01-29 14:57:37,spring,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,full stack Java


In [51]:
#Looking at 46
user_46 = active_df[active_df.id == 46]
user_46.page.value_counts()

javascript-i/bom-and-dom/bom                                                 7
javascript-i/bom-and-dom/dom                                                 7
javascript-i                                                                 5
javascript-i/google-maps-api                                                 4
javascript-i/bom-and-dom/dom-events                                          4
javascript-i/functions                                                       2
javascript-i/loops                                                           2
jquery                                                                       2
javascript-i/objects/math                                                    2
javascript-i/coffee-project                                                  2
javascript-i/javascript-with-html                                            1
jquery/events                                                                1
jquery/selectors                                    

In [52]:
user_46

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 16:30:14,javascript-i/javascript-with-html,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-26 16:30:36,javascript-i/introduction/working-with-data-ty...,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-29 09:33:02,javascript-i/functions,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-29 09:33:04,javascript-i/conditionals,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-29 16:45:43,javascript-i/loops,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-30 09:17:51,javascript-i/functions,46,24.27.227.0,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-30 10:01:40,javascript-i/loops,46,24.27.227.1,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-31 11:04:16,javascript-i/arrays,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-31 16:14:14,javascript-i/arrays/manipulating,46,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java


In [53]:
#Looking at 543
user_543 = active_df[active_df.id == 543]
user_543.page.value_counts()

html-css/css-i/introduction                    8
search/search_index.json                       7
spring/fundamentals/repositories               7
spring/fundamentals/security/authentication    3
html-css/css-ii/bootstrap-introduction         3
spring/fundamentals/relationships              3
spring/fundamentals/form-model-binding         3
spring/fundamentals/views                      3
spring/fundamentals/security                   2
spring/fundamentals/controllers                2
spring/fundamentals/services                   2
spring                                         1
spring/fundamentals/integration-tests          1
appendix/capstone-workbook                     1
html-css/css-ii/media-queries                  1
javascript-i/arrays                            1
javascript-i/objects/math                      1
javascript-i/conditionals                      1
javascript-i/loops                             1
Name: page, dtype: int64

In [54]:
user_543

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2019-11-12 15:05:23,spring/fundamentals/security,543,97.105.19.58,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-12 15:05:32,spring/fundamentals/security/authentication,543,97.105.19.58,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-12 20:39:11,search/search_index.json,543,70.125.148.96,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-12 20:39:25,html-css/css-ii/bootstrap-introduction,543,70.125.148.96,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-12 21:11:12,html-css/css-ii/bootstrap-introduction,543,70.125.148.96,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-13 08:57:02,search/search_index.json,543,97.105.19.58,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-13 08:58:55,spring/fundamentals/repositories,543,97.105.19.58,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-13 08:59:02,spring/fundamentals/relationships,543,97.105.19.58,Ceres,2019-07-15,2019-12-11,149 days,full stack Java
2019-11-13 08:59:11,spring/fundamentals/form-model-binding,543,97.105.19.58,Ceres,2019-07-15,2019-12-11,149 days,full stack Java


## 2. Is there a cohort that referred to a lesson significantly more, that other cohorts seemed to gloss over?

In [56]:
#Aggregate by Cohort name and Page. Make into a dataframe
page_by_cohort = student_df.groupby(['cohort_name'])['page'].value_counts()
page_by_cohort = pd.DataFrame(page_by_cohort)

In [57]:
# Make df readable
page_by_cohort.columns = ['page_count']
page_by_cohort = page_by_cohort.reset_index()
page_by_cohort = page_by_cohort[page_by_cohort['page'] != '/']
page_by_cohort = page_by_cohort.set_index('cohort_name')
page_by_cohort.head()

,page,page_count
cohort_name,,
Andromeda,toc,638
Andromeda,javascript-i,516
Andromeda,spring,487
Andromeda,java-iii,433
Andromeda,java-ii,398


In [63]:
max_page_by_cohort = page_by_cohort.groupby('cohort_name').max()

In [64]:
max_page_by_cohort

,page,page_count
cohort_name,,
Andromeda,web-design/ux/purpose,638
Apex,web-design/ux/purpose,1497
Apollo,content/html-css/introduction.html,2
Arches,uploads/5762c2946250b.jpg,294
Badlands,toc,6
Bash,web-dev-day-two,660
Bayes,viz.py,650
Betelgeuse,web-design/ux/purpose,761
Ceres,web-design/ux/purpose,1380


In [65]:
curriculum_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java


In [66]:
curriculum_df.program_name.value_counts()

full stack Java    713365
data science       103412
full stack PHP      30548
front end               5
Name: program_name, dtype: int64

In [67]:
#Create dataframes for Data Science and Web development
#DS program dataframe
ds_df = curriculum_df[(curriculum_df.program_name == 'data science') & (curriculum_df.cohort_name != 'Staff')]
#Web Dev/PHP dataframe
web_df = curriculum_df[(curriculum_df.program_name != 'data science') & (curriculum_df.cohort_name != 'Staff')]

In [68]:
ds_df

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2019-08-20 09:39:58,/,466,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:39:59,/,467,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:39:59,/,468,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:40:02,/,469,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:40:08,/,470,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
...,...,...,...,...,...,...,...,...
2021-04-21 16:14:16,regression/project,841,99.162.244.233,Easley,2020-12-07,2021-06-08,183 days,data science
2021-04-21 16:15:51,regression/project,841,99.162.244.233,Easley,2020-12-07,2021-06-08,183 days,data science
2021-04-21 16:23:53,/,143,173.174.194.60,Easley,2020-12-07,2021-06-08,183 days,data science


In [69]:
web_df

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
...,...,...,...,...,...,...,...,...
2021-04-21 16:36:09,jquery/personal-site,869,136.50.98.51,Marco,2021-01-25,2021-07-19,175 days,full stack Java
2021-04-21 16:36:34,html-css/css-ii/bootstrap-grid-system,948,104.48.214.211,Neptune,2021-03-15,2021-09-03,172 days,full stack Java
2021-04-21 16:37:48,java-iii,834,67.11.50.23,Luna,2020-12-07,2021-06-08,183 days,full stack Java


In [71]:
#Get max pages by cohort for all
max_page_by_cohort_df = pd.DataFrame()
for i in max_page_by_cohort.index:
    df1 = page_by_cohort.loc[i][page_by_cohort.loc[i]['page_count'] == (max_page_by_cohort.loc[i]['page_count'])]
    max_page_by_cohort_df = pd.concat([max_page_by_cohort_df, df1])
max_page_by_cohort_df

,page,page_count
cohort_name,,
Andromeda,toc,638
Apex,search/search_index.json,1497
Apollo,content/html-css,2
Arches,javascript-i,294
Badlands,content/php_ii/command-line,6
Bash,search/search_index.json,660
Bayes,1-fundamentals/modern-data-scientist.jpg,650
Betelgeuse,search/search_index.json,761
Ceres,search/search_index.json,1380


### Going to investigate Several lessons that appear and see the value counts for each (DS and WD) to see if they are truly anomalies

In [72]:
ds_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2019-08-20 09:39:58,/,466,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:39:59,/,467,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:39:59,/,468,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:40:02,/,469,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science
2019-08-20 09:40:08,/,470,97.105.19.58,Bayes,2019-08-19,2020-01-30,164 days,data science


In [75]:
ds_df[ds_df.page == '1-fundamentals/modern-data-scientist.jpg'].groupby['cohort_name']

TypeError: 'method' object is not subscriptable

In [77]:
#looking in page by cohort to make comparison modern data science jpg
page_by_cohort[(page_by_cohort.index.isin(ds_df.cohort_name)) & (page_by_cohort.page == '1-fundamentals/modern-data-scientist.jpg')]

,page,page_count
cohort_name,,
Bayes,1-fundamentals/modern-data-scientist.jpg,650
Curie,1-fundamentals/modern-data-scientist.jpg,467
Darden,1-fundamentals/modern-data-scientist.jpg,470
Easley,1-fundamentals/modern-data-scientist.jpg,7
Florence,1-fundamentals/modern-data-scientist.jpg,61


In [87]:
#Looking at classification overview for Data Science
page_by_cohort[(page_by_cohort.index.isin(ds_df.cohort_name)) & (page_by_cohort.page == 'classification/overview')]

,page,page_count
cohort_name,,
Bayes,classification/overview,35
Curie,classification/overview,136
Darden,classification/overview,1109
Easley,classification/overview,445
Florence,classification/overview,60


### For Data Science, Darden cohort looked at classification overview 1109 times, while Bayes cohort looked at classification overview 35 times. The other DS cohorts range from 60-445 times.

-----------

### Web Dev/PHP Cohort page views

In [92]:
#looking in page by cohort to make comparison java iii
page_by_cohort[(page_by_cohort.index.isin(web_df.cohort_name)) & (page_by_cohort.page == 'java-iii')]

,page,page_count
cohort_name,,
Andromeda,java-iii,433
Apex,java-iii,663
Arches,java-iii,153
Bash,java-iii,324
Betelgeuse,java-iii,472
Ceres,java-iii,682
Deimos,java-iii,523
Europa,java-iii,451
Fortuna,java-iii,786


### I've tested many of these for Web Dev/PHP and nothing really stands out as being accessed much more/much less than other curriculum items. Changes in naming of course modules also created more unique curriculum items that were the same/similar even though the page was different